# Ingest Darwin Core OBIS ENV formatted data for MBON portal display

## An overview

The Marine Biodiversity Observation Network portal is based on a multi-layer technology stack.  It involves web design, and of course data science type technical solutions, including but not limited to tools like: geoserver, wms, wfs, and javascript.

The data provided by MBON or affiliated researchers is often in a community-held standard, Darwin Core, specifically an application of Darwin Core that enables OBIS's large, interoperable database. 

But these datasets need to be made table-like and lighter weight for optimal portal-map functionality. It's a balancing act between displaying useful data products, and giving user's a satisfactory level of functionality.

## The steps

1) Get the data from a provider - sometimes directly, usually through the OBIS endpoint.

2) Look at the data and metadata, and together with other stakeholders, determine the visualization goals/requirements of this dataset.

3) Download and wrangle the data into 'wide' or 'table-like' shape. (Darwin Core in OBIS-ENV format is 'long' or 'tidy')

This involves:

    - Determining the 'keys' that link events/occurrence and measurements across the 3 tables of OBIS-ENV format
    
    - Determining Null data, some data clean up like trimming whitespace or trailing characters, dropping columns unnecessary for visualization.
    
    - Joining tables, Occurrence, Event and MeasurementOrFact into a single table.
    
4) Formatting and typing the date-time field and other 'cleaning' peices of the final data table.
5) Sending the table to the server that will host it for visualization at a moment's notice on the MBON Portal.
6) The map in the data portal performs many client-side tasks as well, that create the final product on the map, tasks like sum, count, or assign a color palette to the 'heatmap' effect.

 ## This notebook...
 ... walks through a generic set of steps that represent this process with more detail. This notebook contains almost all the same steps and manipualtions done to the data through it's journey to the map.
 
 By: Adrienne Canino, Axiom Data Science  
 June 1, 2022  
 Version: 0.5

# Setup

In [ ]:
#setup environment
import pandas as pd # a data wrangling/analytics library
import geopandas as gpd # a library for geospatial data transformations
from shapely.geometry import Point #another library for geospatial data transformations

In [1]:
#Pull in data - these data are usually stored locally to the notebook, so it's a bit of looking around with ls and cd line commands

%ls


Example_OBIS_INGEST.ipynb


# Wrangle

Each of the occurrence, event and mof tables get pulled into their own data frames (a table for manipulating the data in this analysis environment)

In [ ]:
occ = pd.read_csv('occurrence.txt', sep="\t")
print(occ.columns)
mof = pd.read_csv('extendedmeasurementorfact.txt', sep="\t")
print(mof.columns)
event = pd.read_csv('event.txt', sep="\t")
print(event.columns)

## Look at the tops and tails of the data tables, find how many unique values are there

I may look for specific values or just poke around.

In [ ]:
print(mof['measurementType'].unique())
fish_mof.head()


In [ ]:
#look at the top
occ.head()

In [ ]:
event.head()

In [ ]:
#looking for what measurements are in the dataset
mof['measurementType'].unique()

In [ ]:
#looking at the observations of a specific measurement
mof.loc[mof['measurementType']== "Biomass"]

In [ ]:
#unerstand the dataframe's shape, nulls, data types, length, etc
occ.info()

In [ ]:
event.info()

## Check for those 'key' ids that should create join-locations across the three tables to one single table

Hope for no whammys.

In [ ]:
occID = occ['id'].unique()
eventID = event['id'].unique()
mofID = mof['id'].unique()
occID == eventID #Trues
eventID ==mofID #Trues

# Begin joining tables



## Join event and occurrence
The event table, and the occurrence table, can be joined together on `occurrenceID` without any further transformation (usually). 

In [ ]:
#make a new dataframe (df) with occurrence
df = occ.set_index('id').join(event.set_index('id'), on="id", rsuffix="_event") 
# I specified the index to avoid an error about 'type' of columns
#now take a look, because just the assigning doesn't give me an output cell
df.head()

In [ ]:
# I really like to see the shape and all of the dataframe often
df.info()

In [ ]:
#And I like to see the columns exactly for the next step, dropping columns I will not need for the portal's visualzation
df.columns

In [ ]:
#I often check if there are more than one values for variables that could be of interest in the visualization,
#but if there is only one single unique value, I will often remove the column to reduce load time and complexity of the final map
df['waterBody'].unique()

### Trim that intermediary dataframe

In [ ]:
#Trim the data frame by dropping the columns not needed for visualization.
df.drop(columns=['eventID_event','eventDate_event',
                'decimalLatitude_event', 'decimalLongitude_event', 'verbatimEventDate','year',
                'taxonomicStatus', 'acceptedNameUsage','phylum', 'class','order', 'family', 'genus',
                'specificEpithet','scientificNameAuthorship','recordedBy','identifiedBy','identificationRemarks', 
                'geodeticDatum','coordinateUncertaintyInMeters', 'georeferencedBy','georeferenceProtocol',
                'modified', 'language', 'license', 'references', 'dynamicProperties',
                 'samplingProtocol', 'sampleSizeValue', 'sampleSizeUnit','dataGeneralizations',
                 'institutionID', 'institutionCode', 'datasetName', 'ownerInstitutionCode',
                'language', 'license', 'references', 'institutionID', 'waterBody', 'locality',
                 'country', 'countryCode', 'stateProvince'
                ])

## Pivot the 'long' style MOF table

In [ ]:
# taking a look
mof.head()

In [ ]:
#checking for expected values to be their own new columns
mof['measurementType'].unique()

In [ ]:
mof['measurementMethod'].unique()

In [ ]:
#I will often also check for specifics, sometimes only to better understand the data I'm trying to manipulate, sometimes to answer questions as I see inconcsistencies or challenges to my final objective
mof.loc[mof['measurementType']== "Biomass"]

In [ ]:
# test the pivot
mof.pivot(index='occurrenceID', columns='measurementType', values='measurementValue')
#if that looks like what I want, assign it to a new dataframe

In [ ]:
#assign the pivoted, or 'wide', table to a dataframe of it's own
mofDF = fish_mof.pivot(index='occurrenceID', columns='measurementType', values='measurementValue')
mofDF.head()

In [ ]:
#reset the column names and index and everything so it's a well behaved data frame
mofDF = mofDF.rename_axis(None, axis=1).reset_index('occurrenceID')


In [ ]:
#check that it worked
mofDF.head()

## Join the intermediary data frames together
All wide tables now making one big wide table.

In [ ]:
df = df.join(mofDF.set_index('occurrenceID'), on="occurrenceID")
df.head()
#boom all three joined

In [ ]:
#get my columns again, getting ready to trim the tables
df.columns

# Set up for final visualization

Listing and determining the final columns I'll need for visualizing these data on the portal map.
It's usually a list like this:
* scientificName 
* aphiaID
* lat/lon GEOM
* eventID
* eventDate
* occurrenceID
* occurrenceStatus
* measurements
   - Biomass
   - Size

Sometimes there are additional elements of interest, depending on the dataset.

In [ ]:
# Trim the data frame again down to the very basics
df = df.drop(columns=['basisOfRecord', 'recordedBy', 'individualCount', 'decimalLatitude',
       'decimalLongitude', 'identifiedBy', 'identificationRemarks',
       'scientificNameID', 'acceptedNameUsage', 'kingdom',
       'phylum', 'class', 'order', 'family', 'genus', 'specificEpithet',
       'scientificNameAuthorship', 'vernacularName', 'taxonomicStatus', 'type',
       'modified', 'language', 'license', 'references', 'institutionID',
       'datasetID', 'institutionCode', 'datasetName', 'ownerInstitutionCode',
       'dataGeneralizations', 'dynamicProperties', 'eventID_event',
       'samplingProtocol', 'sampleSizeValue', 'sampleSizeUnit',
       'eventDate_event', 'year', 'verbatimEventDate', 'habitat', 'locationID',
       'waterBody', 'islandGroup', 'country', 'countryCode',
       'stateProvince', 'locality', 'minimumDepthInMeters',
       'maximumDepthInMeters', 'geodeticDatum',
       'coordinateUncertaintyInMeters', 'georeferencedBy',
       'georeferenceProtocol', 'Consumer Type', 'LW_a', 'LW_b', 'Size Bin', 'Trophic Level'])


In [ ]:
df.head()

### Clean up columns

In [ ]:
#clean up taxonID column
df['taxonID'] = df['taxonID'].str.lstrip('aphiaID_')


In [ ]:
df.head()

In [ ]:
#sometimes check and address any value inconsistencies
df['occurrenceStatus'].unique() #there are no caps or etc to mess with category. 


In [ ]:
#check to data types to be correct, most particularly datetimes.
df.info()

# Create geo- dataframe

A dataframe that plays nicely with geospatial data and technologies

In [ ]:
#build geometry part
df['geometry'] = list(zip(df['decimalLongitude_event'], df['decimalLatitude_event']))
df['geometry'] = df['geometry'].apply(Point)
df.head()

In [ ]:
#make geodataframe out of the table, with CRS and geometry explicit
gdf = gpd.GeoDataFrame(df, crs = 'EPSG:4326', geometry='geometry')


In [ ]:
# drop unecessary columns, address any date typing necessary
gdf = gdf.drop(columns=['decimalLongitude_event', 'decimalLatitude_event'])
gdf = gdf.astype({'eventDate':'datetime64[ns]'})
gdf.info()

In [ ]:
# I often save a csv of the table I send to the portal's server for reference in future parts of the process
gdf.to_csv("New_MBON_data_Geoserver.csv")

In [ ]:
#push the table to geoserver database with the right layer name and details

from sqlalchemy import create_engine #I need one more piece of a library to play nice with sql type databases

db_url = 'superURL'

#create_engine from sqlalchemy library
#connects our postgres sql database via its url
engine = create_engine(db_url)


#table name in postgres
layer_name = 'New_MBON_data_2022'

gdf.to_postgis(layer_name, engine, schema='mbon', if_exists='replace', chunksize=1000)

## Pull some metadata 

In [ ]:
# get some details for the geoserver layer etc
print(event['decimalLatitude'].min())
print(event['decimalLatitude'].max())
print(event['decimalLongitude'].min())
print(event['decimalLongitude'].max())

minDate = df['eventDate'].min()
maxDate = df['eventDate'].max()
print("min="+minDate)
print("max="+maxDate)

# Next

Now that the data have been transformed into a light weight table of select variables for visualizing on the portal map, the next steps are to update and attach the Portal catalog information to the server's back end. Then the 'skinning' of the visualization with json and javascript and other more front-end web tools is created, to enable filters, data charts, hover-over features, etc. on the MBON portal. 

Check out the MBON portal for more here: [https://mbon.ioos.us/?ls=s3ShXuOX](https://mbon.ioos.us/?ls=s3ShXuOX)